### Objective：

Design an dimplement a class that will be used to represent bank accounts.

---

### Functionalities and characteristics:

- Accounts are uniquely identified by an **account number** (assume it will just be passed in the initializer)
- Account holders have a **first** and **last** name
- Accounts have an associated **preferred time zone offset** (e.g. -7 for MST)
- The **balances** need to be zero or higher, and should not be directly settable
- But, **deposits** and **withdrawals** can be made (given sufficient funds)
    - if a withdrawal is attempted that would result in nagative funds, the transaction should be declined
- A **monthly interest rate** exists and is applicable to all accounts **uniformly**. There should be a method that can be called to calculate the interest on the current balance using the current interest rate, and **add it** to the balance
- Each deposit and withdrawal must generate a **confirmation number** composed of:
    - the transaction type: `D` for deposit, and `W` for withdrawal, `I` for interest deposit, and `X` for declined (in which case the balance remains unaffected)
    - the account number
    - the time the transaction was made, using UTC
    - an incrementing number (that increments across all accounts and transactions)
    - for (extreme!) simplicity assume that the transaction id starts at zero (or whatever number you choose) whenever the program starts
    - the confirmation number should be returned from any of the transaction methods (deposit, withdraw, etc)
- Create a **method** that, given a confirmation number, returns:
    - the account number, transaction code (D, W, etc), datetime (UTC format), date time (in whatever timezone is specified in te argument, but more human readable), the transaction ID
    - make it so it is a nicely structured object (so can use dotted notation to access these three attributes)
    - I purposefully made it so the desired timezone is passed as an argument. Can you figure out why? (hint: does this method require any information from any instance?)
- Test the code using `unittest` package

---

### Example
- account number `140568` 
- preferred time zone offset of -7 (MST) 
- an existing balance of `100.00`

Suppose the last transaction ID in the system was `123`, and a deposit is made for `50.00` on `2019-03-15T14:59:00` (UTC) on that account (or `2019-03-15T07:59:00` in account's preferred time zone offset)

The new balance should reflect `150.00` and the confirmation number returned should look something like this:

```D-140568-20190315145900-124```

We also want a method that given the confirmation number returns an object with attributes:
- `result.account_number` --> `140568`
- `result.transaction_code` --> `D`
- `result.transaction_id` --> `124`
- `result.time` --> `2019-03-15 07:59:00 (MST)`
- `result.time_utc` --> `2019-03-15T14:59:00`

---
### Implementation

In [1]:
import itertools
import numbers
from datetime import timedelta, datetime

In [2]:
class TimeZone:
    def __init__(self, 
                 name, 
                 offset_hours, 
                 offset_minutes):
        
        if name is None or len(str(name).strip()) == 0:
            raise ValueError('Timezone name cannot be empty.') 
            
        self._name = str(name).strip()
        
        if not isinstance(offset_hours, numbers.Integral):
            raise ValueError('Hour offset must be an integer.')
        
        if not isinstance(offset_minutes, numbers.Integral):
            raise ValueError('Minutes offset must be an integer.')
            
        if offset_minutes < -59 or offset_minutes > 59:
            raise ValueError('Minutes offset must between -59 and 59 (inclusive).')
            
        offset = timedelta(hours=offset_hours, minutes=offset_minutes)

        if offset < timedelta(hours=-12, minutes=0) or offset > timedelta(hours=14, minutes=0):
            raise ValueError('Offset must be between -12:00 and +14:00.')
            
        self._offset_hours = offset_hours
        self._offset_minutes = offset_minutes
        self._offset = offset
        
    @property
    def offset(self):
        return self._offset
    
    @property
    def name(self):
        return self._name
    
    def __eq__(self, other):
        return (isinstance(other, TimeZone) and 
                self.name == other.name and 
                self._offset_hours == other._offset_hours and
                self._offset_minutes == other._offset_minutes)
    def __repr__(self):
        return (f"TimeZone(name='{self.name}', "
                f"offset_hours={self._offset_hours}, "
                f"offset_minutes={self._offset_minutes})")

In [3]:
a = itertools.count(0)

In [4]:
next(a)
next(a)
next(a)
next(a)


3

In [5]:
class BankAccount:
    # Class-level attributes
    _transaction_id = itertools.count(0)
    _interest_rate = 0.5
    _transaction_code = {
        'deposit': 'D',
        'withdraw': 'W',
        'interest': 'I',
        'reject': 'X'
    }
    
    def __init__(self, 
                 account_number,
                 first_name, 
                 last_name, 
                 timezone=None, 
                 starting_balance=0):
        
        self._account_number = account_number
        self.first_name = first_name
        self.last_name = last_name
        
        if timezone==None:
            timezone = TimeZone('UTC', 0, 0)
        self.timezone = timezone
        
        if starting_balance < 0:
            raise ValueError('Balance must be non-negative. ')
        self._balance = float(starting_balance) 
    
    @property
    def account_number(self):
        return self._account_number
    
    @property
    def first_name(self):
        # self._first_name will be set from the setter method
        return self._first_name
    
    @first_name.setter
    def first_name(self, value):
        self._first_name = BankAccount.validate_name(value=value, field='First name')
        
    @property
    def last_name(self):
        return self._last_name
    
    @last_name.setter
    def last_name(self, value):
        self._last_name = BankAccount.validate_name(value=value, field='Last name')
    
    @property
    def full_name(self):
        self._full_name = (' ').join((self._first_name, self._last_name))
        return self._full_name
    
    @staticmethod
    def validate_name(value, field):
        if value is None or len(str(value).strip())==0:
            raise ValueError('{} can\'t be empty.'.format(field))
        return str(value).strip()
      
    @property
    def timezone(self):
        return self._timezone
    
    @timezone.setter
    def timezone(self, value):
        if not isinstance(value, TimeZone):
            raise ValueError('Time zone must be a valid TimeZone object.')
        self._timezone = value
        
    @property
    def balance(self):
        return self._balance
    
    @classmethod
    def interest_rate_getter(cls):
        return cls._interest_rate
    
    @classmethod
    def interest_rate_setter(cls, value):
        if (value < 0):
            raise ValueError('Interest rate must be non-negative.')
        if not isinstance(value, numbers.Real):
            raise ValueError('Interest rate must be a real number.')
        cls._interest_rate = value
        
    def generate_confirmation_code(self, transaction):
        self._confirm_number = '-'.join((self._transaction_code[transaction], 
                                         str(self._account_number), 
                                         datetime.utcnow().strftime('%Y%m%d%H%M%S'), 
                                         str(next(self._transaction_id))))
        
        return self._confirm_number
    
    def deposit(self, value):
        if not isinstance(value, numbers.Real):
            raise ValueError('Deposit must be a real number.')
        if value <= 0:
            raise ValueError('Deposit must be positive.')
        
        self._balance += deposit_value
        return self.generate_confirmation_code('deposit')
    
    def withdraw(self, value):
        if not isinstance(value, numbers.Real):
            raise ValueError('Withdrawl must be a real number.')
        if value <= 0:
            raise ValueError('Withdrawl must be positive.')
            
        if (self._balance - value) < 0:
            return self.generate_confirmation_code('reject')
        else:
            self._balance -= value
            return self.generate_confirmation_code('withdraw')
        
        
    def pay_interest(self):
        self._balance += BankAccount.interest_rate_getter() * self._balance / 100
        return self.generate_confirmation_code('interest')

In [6]:
bank_account = BankAccount(account_number=140568, 
                           first_name='Taylor',
                           last_name='Swift',
                           timezone=None)

---
### Unit test

In [7]:
import unittest

In [8]:
def run_tests(test_class):
    suite = unittest.TestLoader().loadTestsFromTestCase(test_class)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(suite)

<font color='red'> Unit test instruction: </font>
* For all the functions that need to be tested, we should prefix the function name with `test`. 
* Use `setUp` and `tearDown` functions for setting/cleaning up before/after calling each test case.
* Test cases are run in random order.

In [9]:
class TestBankAccount(unittest.TestCase):
    
    def setUp(self):
        self.account_number = 'A100'
        self.first_name = 'FIRST'
        self.last_name = 'LAST'
        self.tz = TimeZone('TZ', 1, 30)
        self.balance = 100
        
    def tearDown(self):
        print('Test is done!')
        
    def test_create_timezone(self):
        tz = TimeZone('ABC', -1, -30)
        self.assertEqual('ABC', tz.name)
        self.assertEqual(timedelta(hours=-1, minutes=-30), tz.offset)
        
    
    def test_timezone_equal(self):
        tz1 = TimeZone('ABC', -1, -30)
        tz2 = TimeZone('ABC', -1, -30)
        self.assertEqual(tz1, tz2)
        
    def test_timezone_unequal(self):
        tz = TimeZone('ABC', -1, -30)
        
        test_timezones = (
            TimeZone('DEF', -1, -30),
            TimeZone('ABC', 1, -30),
            TimeZone('ABC', -1, 0)
        )
        
        for test in test_timezones:
            self.assertNotEqual(tz, test)
        
    def test_create_account(self):
        self.assertEqual(self.account_number, 'A100')
        self.assertEqual(self.first_name, 'FIRST')
        self.assertEqual(self.last_name, 'LAST')
        self.assertEqual(' '.join((self.first_name, self.last_name)), 'FIRST LAST')
        self.assertEqual(self.tz, TimeZone('TZ', 1, 30))
        self.assertEqual(self.balance, 100)
        
    def test_create_account_blank_first_name(self):
        self.first_name = ''
        
        # Under this context manager, given exception should be raised, otherwise the test fails
        with self.assertRaises(ValueError):
            account = BankAccount(account_number=self.account_number,
                                  first_name=self.first_name,
                                  last_name=self.last_name)
            
    def test_create_negative_balance(self):
        self.balance = -100
        
        with self.assertRaises(ValueError):
            account = BankAccount(account_number=self.account_number,
                                  first_name=self.first_name,
                                  last_name=self.last_name,
                                  starting_balance=self.balance)
            
    
    def test_account_withdraw_ok(self):
        account = BankAccount(account_number=self.account_number,
                              first_name=self.first_name,
                              last_name=self.last_name,
                              starting_balance=self.balance)
        
        exe = account.withdraw(20)
        
        self.assertTrue(exe.startswith('W'))
        self.assertEqual(self.balance-20, account.balance)
        
    def test_account_withdraw_overdraw(self):
        account = BankAccount(account_number=self.account_number,
                              first_name=self.first_name,
                              last_name=self.last_name,
                              starting_balance=self.balance)
        
        exe = account.withdraw(200)

        self.assertTrue(exe.startswith('X'))
        self.assertEqual(self.balance, account.balance)

In [10]:
run_tests(TestBankAccount)

test_account_withdraw_ok (__main__.TestBankAccount) ... ok
test_account_withdraw_overdraw (__main__.TestBankAccount) ... ok
test_create_account (__main__.TestBankAccount) ... ok
test_create_account_blank_first_name (__main__.TestBankAccount) ... ok
test_create_negative_balance (__main__.TestBankAccount) ... ok
test_create_timezone (__main__.TestBankAccount) ... ok
test_timezone_equal (__main__.TestBankAccount) ... ok
test_timezone_unequal (__main__.TestBankAccount) ... 

Test is done!
Test is done!
Test is done!
Test is done!
Test is done!
Test is done!
Test is done!
Test is done!


ok

----------------------------------------------------------------------
Ran 8 tests in 0.004s

OK
